In [78]:
import os 
import pandas as pd
import matplotlib.pyplot as plt
os.chdir('/home/s212945/snap/snapd-desktop-integration/current/Documents/Thesis/codeSergio/data')

In [79]:
income_df_code = pd.read_csv('INDKP105_code.csv', sep =';')
income_df_val = pd.read_csv('INDKP105_value.csv', sep =';')

income_df_code['id'] = income_df_code.index
income_df_val['id'] = income_df_val.index

income_df = income_df_code.merge(income_df_val, on='id', suffixes=('_c', '_v'))

mun_cod_val = income_df[['OMRÅDE_v', 'OMRÅDE_c']].drop_duplicates()
income_df.drop(['ENHED_c','ENHED_v', 'TID_c', 'TID_v', 'INDHOLD_v', 'id'], inplace=True, axis=1)

income_df = income_df.loc[income_df.OMRÅDE_v != 'All Denmark']
income_df = income_df.loc[income_df.OMRÅDE_v != 'Region Sjælland']
income_df = income_df.loc[income_df.OMRÅDE_v != 'Region Midtjylland']
income_df = income_df.loc[income_df.OMRÅDE_v != 'Region Syddanmark']
income_df = income_df.loc[income_df.OMRÅDE_v != 'Region Nordjylland']
income_df = income_df.loc[income_df.OMRÅDE_v != 'Region Hovedstaden']
income_df = income_df.loc[income_df.ALDER1_v != '70-74 years']
income_df = income_df.loc[income_df.ALDER1_v != 'More than 74 years']

filter = income_df['OMRÅDE_v'].str.contains('Province')
income_df = income_df[~filter]
income_df.INDHOLD_c.replace(to_replace='..', value='0', inplace=True)
income_df = income_df.rename(columns={'OMRÅDE_c':'Municipality', 'INDKINTB_v':'Income', 'ALDER1_v':'AgeGroup', 'KOEN_c': 'Gender', 'INDHOLD_c':'total'})
income_df.total = income_df.total.astype('int64')
income_df = income_df[['Gender', 'Income','total']]
income_ag = income_df.groupby(['Gender','Income'])['total'].sum()

In [87]:
pop_df = pd.read_csv('pop.csv')
pop_df = pop_df.loc[pop_df.Year == 2021]
pop_df = pop_df.loc[pop_df.Municipality != 'All Denmark']
pop_df = pop_df.loc[pop_df.Municipality != 'Region Sjælland']
pop_df = pop_df.loc[pop_df.Municipality != 'Region Midtjylland']
pop_df = pop_df.loc[pop_df.Municipality != 'Region Syddanmark']
pop_df = pop_df.loc[pop_df.Municipality != 'Region Nordjylland']
pop_df = pop_df.loc[pop_df.Municipality != 'Region Hovedstaden']
pop_df.drop(['Unnamed: 0'], axis='columns', inplace=True)

pop_df = pop_df.rename(columns = {'Val':'total'})
pop_df = pd.merge(pop_df, mun_cod_val, left_on='Municipality', right_on='OMRÅDE_v')

pop_df['Gender'] = pop_df['Gender'].astype('category')
#pop_ag = pop_ag.rename(columns={'OMRÅDE_c':'Municipality'})
pop_df['Gender'] = pop_df.Gender.cat.rename_categories({'Men': 'M', 'Women': 'K'})

#pop_df = pop_df.groupby(['Gender','PopSocio'])['total'].sum()
pop1 = pop_df.copy()
pop2 = pop_df.copy()
pop_df['total'] = pop_df['total'].apply(lambda x: [1] * x)
temp = pop_df.explode('total')
temp
pop_ag = temp.sample(n=income_df.total.sum()).groupby(['Gender', 'PopSocio'])['total'].sum() 

/tmp/ipykernel_4072886/1811272414.py:24: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pop_ag = temp.sample(n=income_df.total.sum()).groupby(['Gender', 'PopSocio'])['total'].sum()


In [93]:
pop1 = pop1.groupby(['Gender', 'Sector'])['total'].sum() 

/tmp/ipykernel_4072886/3852025894.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pop1 = pop1.groupby(['Gender', 'Sector'])['total'].sum()


In [91]:
pd_gen = pd.read_csv("wgan_samples.txt")
pd_gen = pd_gen.rename(columns={'IncomeCat': 'Income', 'MunicipalityOrigin':'Municipality'})
#pd_gen = pd_gen.groupby(['Income', 'Gender', 'PopSocio']).size()

In [92]:
pd_gen = pd_gen.groupby(['Sector', 'Gender', 'PopSocio'], as_index=False).size()
Init = pd_gen.rename(columns={'size': 'total'})

In [25]:
import numpy as np
from ipfn import ipfn

m      = np.array([1., 2., 1., 3., 5., 5., 6., 2., 2., 1., 7., 2.,
               5., 4., 2., 5., 5., 5., 3., 8., 7., 2., 7., 6.], )
dma_l  = [501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501, 501,
          502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502, 502]
size_l = [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4,
          1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4]

age_l  = ['20-25','30-35','40-45',
          '20-25','30-35','40-45',
          '20-25','30-35','40-45',
          '20-25','30-35','40-45',
          '20-25','30-35','40-45',
          '20-25','30-35','40-45',
          '20-25','30-35','40-45',
          '20-25','30-35','40-45']

df = pd.DataFrame()
df['dma'] = dma_l
df['size'] = size_l
df['age'] = age_l
df['total'] = m

xipp = df.groupby('dma')['total'].sum()
xpjp = df.groupby('size')['total'].sum()
xppk = df.groupby('age')['total'].sum()
xijp = df.groupby(['dma', 'size'])['total'].sum()
xpjk = df.groupby(['size', 'age'])['total'].sum()
# xppk = df.groupby('age')['total'].sum()

xipp.loc[501] = 52
xipp.loc[502] = 48

xpjp.loc[1] = 20
xpjp.loc[2] = 30
xpjp.loc[3] = 35
xpjp.loc[4] = 15

xppk.loc['20-25'] = 35
xppk.loc['30-35'] = 40
xppk.loc['40-45'] = 25

xijp.loc[501] = [9, 17, 19, 7]
xijp.loc[502] = [11, 13, 16, 8]

xpjk.loc[1] = [7, 9, 4]
xpjk.loc[2] = [8, 12, 10]
xpjk.loc[3] = [15, 12, 8]
xpjk.loc[4] = [5, 7, 3]

aggregates = [xipp, xpjp, xppk, xijp, xpjk]
dimensions = [['dma'], ['size'], ['age'], ['dma', 'size'], ['size', 'age']]


In [26]:
df

,dma,size,age,total
0,501,1,20-25,1.0
1,501,1,30-35,2.0
2,501,1,40-45,1.0
3,501,2,20-25,3.0
4,501,2,30-35,5.0
5,501,2,40-45,5.0
6,501,3,20-25,6.0
7,501,3,30-35,2.0
8,501,3,40-45,2.0
9,501,4,20-25,1.0


In [27]:
IPF = ipfn.ipfn(df, aggregates, dimensions)
df = IPF.iteration()

print(df)
print(df.groupby('size')['total'].sum(), xpjp)

    size    age  dma      total
0      1  20-25  501   2.155122
1      1  20-25  502   4.844878
2      1  30-35  501   4.738761
3      1  30-35  502   4.261239
4      1  40-45  501   2.106116
5      1  40-45  502   1.893884
6      2  20-25  501   3.792360
7      2  20-25  502   4.207640
8      2  30-35  501   7.204167
9      2  30-35  502   4.795833
10     2  40-45  501   6.003472
11     2  40-45  502   3.996528
12     3  20-25  501  12.031393
13     3  20-25  502   2.968607
14     3  30-35  501   4.035095
15     3  30-35  502   7.964905
16     3  40-45  501   2.933445
17     3  40-45  502   5.066555
18     4  20-25  501   2.026015
19     4  20-25  502   2.973985
20     4  30-35  501   4.037026
21     4  30-35  502   2.962974
22     4  40-45  501   0.936959
23     4  40-45  502   2.063041
size
1    20.0
2    30.0
3    35.0
4    15.0
Name: total, dtype: float64 size
1    20.0
2    30.0
3    35.0
4    15.0
Name: total, dtype: float64


/home/s212945/snap/snapd-desktop-integration/83/Documents/Thesis/.popsyn/lib/python3.10/site-packages/pandas/core/dtypes/common.py:1636: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  npdtype = np.dtype(dtype)
/home/s212945/snap/snapd-desktop-integration/83/Documents/Thesis/.popsyn/lib/python3.10/site-packages/pandas/core/dtypes/common.py:1636: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  npdtype = np.dtype(dtype)
/home/s212945/snap/snapd-desktop-integration/83/Documents/Thesis/.popsyn/lib/python3.10/site-packages/pandas/core/dtypes/common.py:1636: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  npdtype = np.dtype(dtype)
/home/s212945/snap/

In [29]:
agg

[Gender  PopSocio                
 M       Employed                    1320892
         Enrolled in education        304736
         Outside the labour force     346886
         Unemployed                    32535
 K       Employed                    1180608
         Enrolled in education        322967
         Outside the labour force     439115
         Unemployed                    31753
 Name: total, dtype: object,
 Gender  Income               
 K       100,000 - 124,999 DKK     46437
         125,000 - 149,999 DKK     67984
         150,000 - 174,999 DKK     74262
         175,000 - 199,999 DKK     89516
         200,000 - 224,999 DKK     93481
         225,000 - 249,999 DKK    114006
         25,000 - 49,999 DKK       54137
         250,000 - 299,999 DKK    197206
         300,000 - 349,999 DKK    188172
         350,000 - 399,999 DKK    195716
         400,000 - 449,999 DKK    176776
         450,000 - 499,999 DKK    139165
         50,000 - 74,999 DKK       36408
         500,

In [52]:
Init

total
Income                Gender PopSocio       
100,000 - 124,999 DKK K      0            20
                             1            21
                             2             2
                             3           134
                      M      0            35
...                                      ...
Under 25,000 DKK      K      3            73
                      M      0           735
                             1           133
                             2             6
                             3            67

[109 rows x 1 columns]

In [55]:
pd_gen

Income                 Gender  PopSocio
100,000 - 124,999 DKK  K       0            20
                               1            21
                               2             2
                               3           134
                       M       0            35
                                          ... 
Under 25,000 DKK       K       3            73
                       M       0           735
                               1           133
                               2             6
                               3            67
Length: 109, dtype: int64

In [72]:
pop_ag.groupby(['Gender']).sum()

/tmp/ipykernel_4072886/3319069225.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pop_ag.groupby(['Gender']).sum()


Gender
M    2004895
K    1974597
Name: total, dtype: object

In [73]:
income_ag.groupby(['Gender']).sum()

Gender
K    1978398
M    2001094
Name: total, dtype: int64

In [94]:
import numpy as np
from ipfn import ipfn
agg = [pop_ag, pop1]
dim = [['PopSocio', 'Gender'], ['Sector', 'Gender']]

IPF = ipfn.ipfn(Init, agg, dim, convergence_rate = 0.01)
FIT = IPF.iteration()

KeyError: (0, 'K')